In [6]:
!pip3 install -r requirements.txt


You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import csv
import inflect

p = inflect.engine()

def form_singular(word):
    return p.singular_noun(word) or word

occupations = []
salaries = []

with open('datasets/occupation_salaries.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        occupation_cell = row[0] #.strip()
        #num_leading_spaces = len(occupation) - len(occupation.lstrip(' '))
        try:
            salary = float(row[1].replace(',',''))
            if (len(occupation_cell) > 0) and salary:
            #if (num_leading_spaces == 6) and salary:
                #occupation = " ".join(map(form_singular, occupation.split(' ')))
                #occupation = occupation.replace('and', 'or')
                for occupation in occupation_cell.split(','):
                    occupations.append(occupation.strip())
                    salaries.append(salary)
                #print(occupation, salary)
        except ValueError:
            pass
        

#normalised_salaries = [(40 * (float(s) / max(salaries))) for s in salaries]
#print('normalised_salaries ', normalised_salaries )
print(list(zip(salaries, occupations)))
                

[(55745.0, 'Manager'), (55745.0, 'Director'), (55745.0, 'Senior Official'), (60136.0, 'Corporate Manager'), (60136.0, 'Corporate Director'), (122818.0, 'Senior Executive'), (122818.0, 'Senior Official'), (133072.0, 'CEO'), (55029.0, 'Production Manger'), (55029.0, 'Production Director'), (56322.0, 'Manufacturing Director'), (56322.0, 'Manufacturing Manager'), (48796.0, 'Construction Manager'), (48796.0, 'Construction Director'), (55109.0, 'Mining Production Manger'), (55109.0, 'Energy Production Manager'), (73085.0, 'Functional Manager'), (73085.0, 'Functional Director'), (79878.0, 'Financial Manager'), (79878.0, 'Financial Director'), (83144.0, 'Marketing Director'), (83144.0, 'Sales Director'), (55622.0, 'Purchasing Manager'), (55622.0, 'Purchasing Director'), (63461.0, 'Advertising Director'), (63461.0, 'Public Relations Director'), (53359.0, 'HR manager'), (53359.0, 'HR Director'), (82948.0, 'IT Director'), (62232.0, 'Functional Manager'), (76534.0, 'Bank Manger'), (76534.0, 'Bank 

In [2]:
import csv
import math
from os import path
import pandas as pd
from predict import predict
import plotly.express as px
import plotly.graph_objects as go

def get_most_likely_next_words(previous):
    words = [(word, probability) for word, probability in zip(result['words'], result['probabilities'])]
    sorted_words = sorted(words, key=lambda word: word[1])
    return sorted_words

def get_normalised_angle(x, y):
    if y == 0:
        return 1
    else:
        return math.atan(x / y) * 2 / math.pi

def get_cartesian_distance(x, y):
    return math.sqrt(x**2 + y**2)

def get_binary_gender_probabilities(occupation, model_name: str = '345M'):
    ladies_first = predict('Is a {} a woman or a man? A {} is a'.format(occupation, occupation), model_name)
    men_just_before = predict('Is a {} a man or a woman? A {} is a'.format(occupation, occupation), model_name)
    #ladies_first = predict('A {} is a woman or a man. A {} is a'.format(occupation, occupation), model_name)
    #men_just_before = predict('A {} is a man or a woman. A {} is a'.format(occupation, occupation), model_name)
    probability_woman = (ladies_first.get('woman', 0) + men_just_before.get('woman', 0)) / 2
    probability_man = (ladies_first.get('man', 0) + men_just_before.get('man', 0)) / 2
    return {
        'occupation': occupation,
        'probability_woman': probability_woman,
        'probability_man': probability_man,
        'genderedness': get_cartesian_distance(probability_woman, probability_man),
        'maleness': 1 - get_normalised_angle(probability_woman, probability_man),
        'femaleness': get_normalised_angle(probability_woman, probability_man),
        'model_name': model_name
    }

def export_gender_probabilities(occupations, salaries, model_name):
    occupation_gender_dicts = [get_binary_gender_probabilities(occupation, model_name) for occupation in occupations]
    headings = ['occupation',
                'probability_woman',
                'probability_man',
                'genderedness', 
                'femaleness',
                'maleness',
                'salary',
                'model']
    occupation_gender_lines = [[ogd[0].get(headings[0]),
                                ogd[0].get(headings[1]),
                                ogd[0].get(headings[2]),
                                ogd[0].get(headings[3]),
                                ogd[0].get(headings[4]),
                                ogd[0].get(headings[5]),
                                ogd[1],
                                model_name] for ogd in zip(occupation_gender_dicts, salaries)]
    #occupation_genders_345M = [get_binary_gender_probabilities(occupation, '345M') for occupation in occupations]
    print(occupation_gender_lines)
    with open('results/processed_salaries_{}.csv'.format(model_name), 'w') as file:
        writer = csv.writer(file)
        writer.writerows([headings])
        writer.writerows(occupation_gender_lines)
        file.close()
        
export_gender_probabilities(occupations, salaries, '345M')
export_gender_probabilities(occupations, salaries, '117M')
    

/home/ubuntu/lm-explorer/predict.py:15: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



[['Manager', 0.13328854134306312, 0.20639530196785927, 0.24569260454430708, 0.3650452400535264, 0.6349547599464735, 55745.0, '345M'], ['Director', 0.09741150960326195, 0.1720680668950081, 0.19772815138004848, 0.32794614135438965, 0.6720538586456104, 55745.0, '345M'], ['Senior Official', 0.14018461294472218, 0.2064153105020523, 0.24951754670988618, 0.3797993454612184, 0.6202006545387816, 55745.0, '345M'], ['Corporate Manager', 0.14638110902160406, 0.26185160130262375, 0.29998948345424203, 0.32451334780154223, 0.6754866521984577, 60136.0, '345M'], ['Corporate Director', 0.15329049713909626, 0.19714278355240822, 0.24972635747942779, 0.4207470688027063, 0.5792529311972937, 60136.0, '345M'], ['Senior Executive', 0.15080138482153416, 0.22156469896435738, 0.2680148755036905, 0.38044433016145734, 0.6195556698385427, 122818.0, '345M'], ['Senior Official', 0.14018461294472218, 0.2064153105020523, 0.24951754670988618, 0.3797993454612184, 0.6202006545387816, 122818.0, '345M'], ['CEO', 0.1173119344

[['Manager', 0.05167366284877062, 0.15923909842967987, 0.16741343405146938, 0.19976037861653445, 0.8002396213834655, 55745.0, '117M'], ['Director', 0.05100504960864782, 0.09660376980900764, 0.10924194902551108, 0.30925829474361205, 0.690741705256388, 55745.0, '117M'], ['Senior Official', 0.023408446926623583, 0.026569537818431854, 0.035410390105188524, 0.45978757785209695, 0.540212422147903, 55745.0, '117M'], ['Corporate Manager', 0.0181893277913332, 0.060042113065719604, 0.06273680727369878, 0.18726453844547208, 0.8127354615545279, 60136.0, '117M'], ['Corporate Director', 0.02945194812491536, 0.054031409323215485, 0.061537065593068076, 0.3177148093222695, 0.6822851906777305, 60136.0, '117M'], ['Senior Executive', 0.030585430096834898, 0.05451367795467377, 0.06250767647540786, 0.3255003180067021, 0.6744996819932979, 122818.0, '117M'], ['Senior Official', 0.023408446926623583, 0.026569537818431854, 0.035410390105188524, 0.45978757785209695, 0.540212422147903, 122818.0, '117M'], ['CEO', 

In [ ]:

def create_arrow(from_x, from_y, to_x, to_y):
    return go.layout.Annotation(
        x=to_x, y=to_y, xref="x", yref="y", ax=from_x, ay=from_y, axref='x',ayref='y',
        showarrow=True, align="center", arrowhead=2, arrowsize=1, arrowwidth=2, opacity=0.8
    )

def plot_occupations_graph(occupations, salaries=None, file_name=None):
    
    if not salaries:
        salaries = [10 for o in occupations]
        
    occupation_genders_117M = [get_binary_gender_probabilities(occupation, '117M') for occupation in occupations]
    occupation_genders_345M = [get_binary_gender_probabilities(occupation, '345M') for occupation in occupations]
    occupation_genders = occupation_genders_117M + occupation_genders_345M

    plot_occupations = occupations + occupations
    plot_salaries = salaries + salaries

    model_colours = {'117M': 'rgb(215,48,39)', '345M': 'rgb(215,148,39)'}
    colours = [model_colours[g['model_name']] for g in occupation_genders]

    fig = go.Figure()

    fig.update_layout(
        showlegend=False,
        annotations=[
            create_arrow(small['probability_woman'], 
                         small['probability_man'], 
                         medium['probability_woman'], 
                         medium['probability_man']) for small, medium in zip(occupation_genders_117M, occupation_genders_345M)   
        ]
    )

    fig.add_trace(go.Scatter(x=[0,0.3], y=[0,0.3],
                             mode='lines',
                             name='lines'))

    fig.add_trace(go.Scatter(x=[g['probability_woman'] for g in occupation_genders],
                             y=[g['probability_man'] for g in occupation_genders],
                             #mode="markers",
                             mode="markers+text",
                             hovertext=plot_occupations,
                             text=plot_occupations,
                             textfont={
                                #"color": [],
                                #"family": ["Courier New, monospace" for o in plot_occupations],
                                "size": [12 for o in plot_occupations]
                             },
                             textposition=['bottom center' for o in plot_occupations],
                             marker_size=plot_salaries,
                             marker=dict(size=15, color=colours)))
    fig.update_layout(
        #title=go.layout.Title(
        #    text="",
        #    xref="paper",
        #    x=0
        #),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text="Probability Woman",
                font=dict(
                    #family="Courier New, monospace",
                    size=12,
                    color="#7f7f7f"
                )
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text="Probability Man",
                font=dict(
                    #family="Courier New, monospace",
                    size=12,
                    color="#7f7f7f"
                )
            )
        )
    )
    fig.show()

    if file_name:
        with open(path.join('../graphs', file_name), 'w') as file:
            print(fig.to_html(), file=file)

def plot_salaries(occupations, salaries):

    occupation_genders_345M = [get_binary_gender_probabilities(occupation, '345M') for occupation in occupations]
    maleness = (math.atan(1) * 2) / (math.pi)

'''
num_occupations = 50
occupations = occupations[:num_occupations]
salaries = normalised_salaries[:num_occupations]
#plot_occupations_graph(occupations, salaries)

#occupations = ['pilot']
occupations_short = ['pilot', 'surgeon', 'homemaker', 'programmer', 'cleaner', 
                     'president', 'banker', 'footballer', 'politician', 'captain', 'doctor', 'lawyer',
                     'secretary', 'hairdresser', 'model', 'chef', 'cook', 'journalist', 'nurse', 'carer']

plot_occupations_graph(occupations, file_name='test.html')
'''
